In [1]:
import torch

In [2]:
basic_path = "/home1/ohs/cechallenge/ckpt/origin/consolidated.0"

In [3]:
model_lst = []
for i in range(4):
    plus_path = str(i)+".pth"
    model_path = basic_path + plus_path
    model_lst.append(torch.load(f=model_path, map_location="cpu"))

In [4]:
model_weights_name_lst = [x for x in model_lst[0].keys()]

In [5]:
def check_colpar(str):
    column_parallel_name_lst = ["output", "wq", "wk", "wv", "w1", "w3"]
    for colpar_name in column_parallel_name_lst:
        if colpar_name in str:
            return True
    return False

In [6]:
def check_rowpar(str):
    row_parallel_name_lst = ["embeddings", "wo", "w2"]
    for rowpar_name in row_parallel_name_lst:
        if rowpar_name in str:
            return True
    return False

In [7]:
def check_nopar(str):
    no_parallel_name_lst = ["rope", "norm"]
    for nopar_name in no_parallel_name_lst:
        if nopar_name in str:
            return True
    return False

In [8]:
model_weights_dict = {}
for i, model_weights_name in enumerate(model_weights_name_lst):
    
    a = None
    for j in range(3):
        
        if a is None:
            if check_colpar(model_weights_name):
                a = torch.concat((model_lst[j][model_weights_name],
                                  model_lst[j+1][model_weights_name]), dim=0)
            elif check_rowpar(model_weights_name):
                a = torch.concat((model_lst[j][model_weights_name],
                                  model_lst[j+1][model_weights_name]), dim=1)
            elif check_nopar(model_weights_name):
                a = model_lst[j][model_weights_name]
                break
            else:
                assert True, "not matching"
        else:
            if check_colpar(model_weights_name):
                a = torch.concat((a,
                                  model_lst[j+1][model_weights_name]), dim=0)
            elif check_rowpar(model_weights_name):
                a = torch.concat((a,
                                  model_lst[j+1][model_weights_name]), dim=1)
            else:
                assert True, "not matching"
    
    model_weights_dict[model_weights_name] = a

len(model_weights_dict)

544

In [9]:
# torch.save(model_weights_dict,"./whole_model.pt")

In [ ]:
for model_name in model_weights_name_lst:
    print(model_name, model_weights_dict[model_name].shape)